In [1]:
%load_ext lab_black

In [2]:
%config Completer.use_jedi = False

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time
from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
path = "input/"

In [20]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

In [21]:
numerical_cols = [f"cont{i}" for i in range(1, 15)]
target_col = "target"

for c in numerical_cols:
    prep = StandardScaler()
    train[c] = prep.fit_transform(train[[c]])
    test[c] = prep.transform(test[[c]])

X_train = train.drop(["id", "target"], axis=1)
y_train = train["target"]
X_test = test.drop("id", axis=1)

In [24]:
cv = KFold(n_splits=5, shuffle=True, random_state=7)

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    model = keras.Sequential(
        [
            layers.Dense(2 ** 12, activation="relu"),
            layers.Reshape((2 ** 4, 2 ** 8)),
            layers.Conv1D(filters=2 ** 8, kernel_size=3, strides=1, activation="relu"),
#             layers.AvgPool1D(pool_size=2),
#             layers.Conv1D(filters=2 ** 9, kernel_size=3, strides=1, activation="relu"),
            layers.MaxPool1D(pool_size=2),
            layers.Flatten(),
            layers.Dense(2 ** 6, activation="relu"),
            layers.Dense(1, activation="linear"),
        ]
    )

    model.compile(
        optimizer="adam", loss="mse", metrics=[keras.metrics.RootMeanSquaredError()]
    )

    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.05,
        restore_best_weights=True,
    )

    model.fit(
        X_tr,
        y_tr,
        validation_data=(X_val, y_val),
        batch_size=3_000,
        epochs=1_00,
        callbacks=[early_stopping],
    )

    oof_train[valid_index] = model.predict(X_val).reshape(1, -1)[0]
    y_pred = model.predict(X_test).reshape(1, -1)[0]

    y_preds.append(y_pred)
    models.append(model)

Epoch 1/100
80/80 [==============================] - 36s 447ms/step - loss: 14.2024 - root_mean_squared_error: 3.5261 - val_loss: 1.2511 - val_root_mean_squared_error: 1.1185
Epoch 2/100
80/80 [==============================] - 39s 482ms/step - loss: 1.1545 - root_mean_squared_error: 1.0742 - val_loss: 0.9334 - val_root_mean_squared_error: 0.9661
Epoch 3/100
80/80 [==============================] - 38s 476ms/step - loss: 0.8728 - root_mean_squared_error: 0.9341 - val_loss: 0.7327 - val_root_mean_squared_error: 0.8560
Epoch 4/100
80/80 [==============================] - 39s 490ms/step - loss: 0.6881 - root_mean_squared_error: 0.8294 - val_loss: 0.5968 - val_root_mean_squared_error: 0.7726
Epoch 5/100
80/80 [==============================] - 38s 473ms/step - loss: 0.5712 - root_mean_squared_error: 0.7557 - val_loss: 0.5359 - val_root_mean_squared_error: 0.7321
Epoch 6/100
80/80 [==============================] - 39s 484ms/step - loss: 0.5247 - root_mean_squared_error: 0.7243 - val_loss: 

KeyboardInterrupt: 

In [ ]:
models[0].summary()

In [ ]:
print(f"CV: {mean_squared_error(y_train, oof_train, squared=False)}")

In [ ]:
y_pred = sum(y_preds) / len(y_preds)
submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)
submission.head()